In [1]:
# 모듈 import
import os
from mmcv import Config
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor, set_random_seed
from mmseg.datasets import build_dataloader, build_dataset
from mmcv.runner import load_checkpoint

ModuleNotFoundError: No module named 'mmcv'

In [2]:
# config file 들고오기
config = 'upernet_swin_large_patch4_window12_512x512_80k_coco_pretrain_384x384_22K_rmi'
cfg = Config.fromfile(f'./configs/swin/{config}.py')

root = '../input/data/'

cfg.log_config.hooks[1].init_kwargs.name = config
# dataset config 수정
# cfg.data.train.classes = classes
cfg.data.train.data_root = root
cfg.data.train.img_dir = 'train/img/'
cfg.data.train.ann_dir = 'train/ann/'

# cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512)

cfg.data.val.data_root = root
cfg.data.val.img_dir = 'val/img/'
cfg.data.val.ann_dir = 'val/ann/'
# cfg.data.val.pipeline[1]['img_scale'] = (512,512)

# cfg.data.test.classes = classes
cfg.data.test.data_root = root
cfg.data.test.img_dir = 'test/img/'
# cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)

cfg.data.samples_per_gpu = 10
cfg.data.workers_per_gpu = 4

cfg.seed = 1995
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{config}'

# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
# cfg.checkpoint_config = dict(max_keep_ckpts=5, interval=1)
cfg.evaluation['save_best'] = 'mIoU'
# cfg.resume_from = f'./work_dirs/{config}/latest.pth'

In [3]:
# build_dataset
set_random_seed(cfg.seed)
datasets = [build_dataset(cfg.data.train)]

2021-11-05 00:18:46,381 - mmseg - INFO - Loaded 3383 images


In [4]:
# dataset 확인
datasets[0]

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_segmentor(cfg.model)
model.init_weights()

# ckpt_name = 'best_mIoU_iter_58000'
# checkpoint_path = os.path.join(cfg.work_dir, f'{ckpt_name}.pth')
# load_checkpoint(model, checkpoint_path, map_location='cpu')

/opt/ml/segmentation/mmsegmentation/mmseg/models/backbones/swin.py:556: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
2021-11-05 00:18:48,980 - mmseg - INFO - Use load_from_http loader
2021-11-05 00:18:50,300 - mmseg - INFO - initialize UPerHead with init_cfg {'type': 'Normal', 'std': 0.01, 'override': {'name': 'conv_seg'}}
2021-11-05 00:18:50,601 - mmseg - INFO - initialize FCNHead with init_cfg {'type': 'Normal', 'std': 0.01, 'override': {'name': 'conv_seg'}}
2021-11-05 00:18:50,604 - mmseg - INFO - 
backbone.patch_embed.projection.weight - torch.Size([192, 3, 4, 4]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2021-11-05 00:18:50,604 - mmseg - INFO - 
backbone.patch_embed.projection.bias - torch.Size([192]): 
Initialized by user-defined `init_weights` in SwinTransformer  
 
2021-11-05 00:18:50,605 - mmseg - INFO - 
backbone.patch_embed.norm.weight - torc

In [6]:
# 모델 학습
# wandb.init(project='Object_Detection', entity='jujoo')
train_segmentor(model, datasets[0], cfg, distributed=False, validate=True)

2021-11-05 00:18:58,274 - mmseg - INFO - Loaded 655 images
2021-11-05 00:18:58,276 - mmseg - INFO - Start running, host: root@d40555fd3b37, work_dir: /opt/ml/segmentation/mmsegmentation/work_dirs/upernet_swin_large_patch4_window12_512x512_80k_coco_pretrain_384x384_22K_rmi
2021-11-05 00:18:58,277 - mmseg - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) PolyLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) PolyLrUpdaterHook

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
ERROR:root:dropped chunk 404 Client Error: Not Found for url: https://api.wandb.ai/files/jujoo/Segmentation/2s8vrv4j/file_stream
NoneType: None
